### Codice utlizzato per la Deep Neural Network applicato al  nostro caso di studio

Per tentare di ridefinire gli insiemi delle classi abbiamo utilizzato un approccio _unsupervised_. Abbiamo lasciato che le immagini fossero elaborate da un _autoencoder_ [2](#cite-SCHMIDHUBER201585) e ridistribuite secondo un associazione morfologica.

L'utilizzo degli autoencoder è largamente usato nell'approccio non supervisionato. L'obiettivo di questo "oggetto" è dapprima tentare di ridurre la dimensionalità dei dati che cerca di esaminare e poi provare a ricostruire questi ultimi misurando alla fine quanto la ricostruzione si discosta dal dato reale.

![autoencoder](https://upload.wikimedia.org/wikipedia/commons/2/28/Autoencoder_structure.png)

Questo tipo di strutture sono spesso impiegate nei sistemi che devono fornire risposte in tempi relativamente brevi, come ad esempio il riconoscimento dei volti nelle immagini; inoltre, il modello creato potrebbe essere utilizzato per una valutazione parallela rispetto a quella dell'esperto per valutarne la bontà delle scelte.

Nel nostro caso specifico abbiamo sfruttato pienamente la parte di encoder per ridurre la dimensionalità [1](#cite-hinton2006reducing) delle nostre immagini, in modo da risaltarne pienamente le caratteristiche morfologiche per ciascuna di esse. L'uscita di questo è stata posta in ingresso ad un _clusterizzatore_ classico che sfruttasse la tecnica del *k_means*.

In [1]:
# standard libraries
import os
import shutil

import numpy as np
from keras.callbacks import TensorBoard
from keras.callbacks import EarlyStopping as ES
# keras libraries
from keras.layers import Input, Dense, Dropout, Activation
from keras.models import Model, Sequential
import imageio
# sklearn libraries
from sklearn.cluster import KMeans
from sklearn.metrics import confusion_matrix, classification_report
from sklearn import mixture
import matplotlib.image as mpimg

Using TensorFlow backend.


In [6]:
# path to save files
path_dataset = './crop_r_uns'  # dataset base path
path_img_out = './img_cluster_uns'  # path to save clustered images
path_model = './model_uns/'  # path to save autoencoder model
path_board = './tensorboard/'

# unsupervised neural network params
batch_size = 32  # training cases batch
num_epochs = 500  # max number of epochs
out_encoder = 4  # number of neuron of encoder layer
seed = 42   # base random seed

# dataset image parameters
height = 32
width = 32
depth = 1

In [7]:
print('STARTING FITTING UNSUPERVISED NEURAL NETWORK')
if not os.path.exists(path_model):
    os.mkdir(path_model)
if os.path.exists(path_board):
    shutil.rmtree(path_board)
    os.mkdir(path_board)
X_train, Y_train, filenames= _load_data_uns()
print('loading data .........')
num_classes = np.unique(Y_train).shape[0]

STARTING FITTING UNSUPERVISED NEURAL NETWORK
loading data .........


Abbiamo definito una funzione che generasse un nuovo dataset di immagni a partire dalla clusterizzazione, inoltre, per testare e valutare il risultato, abbiamo fatto in modo che le immagini fossero salvate nel nuovo path con il riferimento alla vecchia classe di appartenenza.

In [13]:
def _save_clustering(X, kmeans, path, filenames, predict=False):

    names = []
    for i in range(len(filenames)):
        names.append(filenames[i][20:-4])
    if predict:
        idx0 = np.where(kmeans == 0)
        idx1 = np.where(kmeans == 1)
        idx2 = np.where(kmeans == 2)
    else:
        idx0 = np.where(kmeans.labels_ == 0)
        idx1 = np.where(kmeans.labels_ == 1)
        idx2 = np.where(kmeans.labels_ == 2)

    image0 = X[idx0]
    image1 = X[idx1]
    image2 = X[idx2]

    if os.path.exists(path):
        shutil.rmtree(path)
    i = 0
    os.mkdir(path)
    os.mkdir(path + "/0/")
    for im in image0:
        p = path + "/0/" + names[idx0[0][i]] + ".png"
        image = np.asarray(im).reshape(height, width)
        imageio.imwrite(p, image)
        i = i + 1
    i = 0
    os.mkdir(path + "/1/")
    for im in image1:
        p = path + "/1/" + names[idx1[0][i]] + ".png"
        image = np.asarray(im).reshape(height, width)
        imageio.imwrite(p, image)
        i = i + 1
    i = 0
    os.mkdir(path + "/2/")
    for im in image2:
        p = path + "/2/" + names[idx2[0][i]] + ".png"
        image = np.asarray(im).reshape(height, width)
        imageio.imwrite(p, image)
        i = i + 1

In [14]:
def _load_data_uns():
    """
    load dataset from path
    :return: train dataset and file names
    """
    filenames, labels = generate_dataset(path_dataset)
    X_train = np.asarray(load_image(filenames))
    Y_train = labels

    return X_train, Y_train, filenames

def load_image(filenames):
    """
    load images from and array of path
    :param filenames: array of file names to load
    :return: an array of images
    """
    images = [*map(lambda x: np.asarray([read_image(x)]).reshape((32, 32, 1)), filenames)]
    return images

def read_image(filename):
    """
    load image from disk
    :param filename: path of images
    :return: image match to file name path
    """
    return mpimg.imread(filename)

def iterate_path(path):
    """
    iterate path and sub path to generate an two array of images and labels
    :param path: base path to investigate
    :return: two array of file names and labels
    """
    filenames_array = []
    labels_array = []
    for root, dirs, files in os.walk(path):
        for name in files:
            filenames_array.append(os.path.join(root, name))
            labels_array.append(int(os.path.basename(os.path.normpath(root))))
    # labels_array = _one_hot_label(labels_array)
    return filenames_array, labels_array


def generate_dataset(base_path):
    """
    produce a dataset for neural network from path
    :param base_path: base path of dataset images
    :return: two array with file name and labels
    """
    filenames_array, labels_array = iterate_path(base_path)
    return filenames_array, labels_array


In [9]:
autoencoder = Sequential()
autoencoder.add(Dense(8, activation='sigmoid', input_shape=(height*width,)))
autoencoder.add(Dense(out_encoder))
out_e = Activation('sigmoid')  # output encoder
autoencoder.add(out_e)
autoencoder.add(Dense(8, activation='sigmoid'))
autoencoder.add(Dropout(0.5, seed=seed))
autoencoder.add(Dense(height*width, activation='sigmoid'))

# model generation
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')
encoder = Model(autoencoder.input, out_e.output)
print(autoencoder.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 8)                 8200      
_________________________________________________________________
dense_6 (Dense)              (None, 4)                 36        
_________________________________________________________________
activation_2 (Activation)    (None, 4)                 0         
_________________________________________________________________
dense_7 (Dense)              (None, 8)                 40        
_________________________________________________________________
dropout_2 (Dropout)          (None, 8)                 0         
_________________________________________________________________
dense_8 (Dense)              (None, 1024)              9216      
Total params: 17,492
Trainable params: 17,492
Non-trainable params: 0
_________________________________________________________________
None


In [10]:
# reshape input
X_train_r = np.asarray(X_train).reshape(-1, height*width)
# print(X_train.shape[0])
# batch_size=1
# start fitting process
autoencoder.fit(X_train_r, X_train_r,
                epochs=num_epochs,
                batch_size=batch_size,
                shuffle=True,
                validation_split=0.1,
                verbose=2,
                callbacks=[TensorBoard(log_dir='./uns_deep/tensorboard', histogram_freq=1, write_images=True,
                                       write_grads=True),
                           ES(monitor='val_loss', min_delta=0.001, patience=3,
                                         verbose=0, mode='auto')]
)

autoencoder.save(path_model+'autoencoder.h5')

encoded_img = encoder.predict(X_train_r)

Train on 4744 samples, validate on 528 samples
Epoch 1/500
 - 1s - loss: 0.6737 - val_loss: 0.6519
Epoch 2/500
 - 1s - loss: 0.6273 - val_loss: 0.6014
Epoch 3/500
 - 1s - loss: 0.5766 - val_loss: 0.5533
Epoch 4/500
 - 1s - loss: 0.5369 - val_loss: 0.5189
Epoch 5/500
 - 1s - loss: 0.5104 - val_loss: 0.4970
Epoch 6/500
 - 1s - loss: 0.4932 - val_loss: 0.4830
Epoch 7/500
 - 1s - loss: 0.4830 - val_loss: 0.4739
Epoch 8/500
 - 1s - loss: 0.4767 - val_loss: 0.4676
Epoch 9/500
 - 1s - loss: 0.4694 - val_loss: 0.4633
Epoch 10/500
 - 1s - loss: 0.4669 - val_loss: 0.4601
Epoch 11/500
 - 1s - loss: 0.4647 - val_loss: 0.4577
Epoch 12/500
 - 1s - loss: 0.4626 - val_loss: 0.4560
Epoch 13/500
 - 1s - loss: 0.4613 - val_loss: 0.4546
Epoch 14/500
 - 1s - loss: 0.4600 - val_loss: 0.4535
Epoch 15/500
 - 1s - loss: 0.4589 - val_loss: 0.4527
Epoch 16/500
 - 1s - loss: 0.4576 - val_loss: 0.4520
Epoch 17/500
 - 1s - loss: 0.4574 - val_loss: 0.4514
Epoch 18/500
 - 1s - loss: 0.4569 - val_loss: 0.4509
Epoch 19

Utilizziamo il *k_means* per clusterizzare l'uscita del nostro encoder imponendo come numero di cluster il valore a noi noto delle classi che vorremmo in uscita, valore pari a 3.

In [15]:
kmeans_all = KMeans(n_clusters=num_classes, init='k-means++', random_state=seed)\
        .fit_predict(np.asarray(encoded_img).reshape(-1, out_encoder))
unique_all, counts_all = np.unique(kmeans_all, return_counts=True)

Il risultato del nostro clusterizzatore lo sfruttiamo per ricreare un nuovo dataset nel quale le immagini sono ricollocate nella classe di appartenenza.

Al termine di questo task proviamo a stimare quanto la classificazione iniziale sia _distante_ rispetto a quella prodotta dal nostro sistema non supervisionato.

In [12]:
order_class = np.sort(counts_all)[::-1]
ordered_v = np.empty(shape=kmeans_all.shape)
if (counts_all != order_class).all:
    idx_0 = np.asscalar(np.asarray(np.where(counts_all == order_class[0])))
    idx_1 = np.asscalar(np.asarray(np.where(counts_all == order_class[1])))
    idx_2 = np.asscalar(np.asarray(np.where(counts_all == order_class[2])))
    ordered_v[np.asarray(np.where(kmeans_all == idx_0))] = 0
    ordered_v[np.asarray(np.where(kmeans_all == idx_1))] = 1
    ordered_v[np.asarray(np.where(kmeans_all == idx_2))] = 2

unique_ord, counts_ord = np.unique(ordered_v, return_counts=True)
_save_clustering(X=X_train_r, kmeans=ordered_v, path=path_img_out, filenames=filenames,
                 predict=True)

print('\nOrdered Count : ', counts_ord)

confmatrix = confusion_matrix(Y_train, ordered_v)
print("\nConfusion Matrix :")
print(confmatrix)

class_names = ["0", "1", '2']

print('\nClassification Report : ')
print(classification_report(Y_train, ordered_v, target_names=class_names))

/home/claudio/Documents/oil_spill/lib/python3.5/site-packages/imageio/core/util.py:78: UserWarning: Lossy conversion from float32 to uint8, range [0, 1]
  dtype_str, out_type.__name__))



Ordered Count :  [3197 1618  457]

Confusion Matrix :
[[2999 1552  290]
 [  51   21   25]
 [ 147   45  142]]

Classification Report : 
             precision    recall  f1-score   support

          0       0.94      0.62      0.75      4841
          1       0.01      0.22      0.02        97
          2       0.31      0.43      0.36       334

avg / total       0.88      0.60      0.71      5272



Come ci aspettavamo, i risultati sono abbastanza vicini per gli elementi di classe 0 (low risk), mentre sono molto distanti per gli elementi di classe 1 (medium risk). Per questa classe si passa addirittura da alcune centinaia di elementi a ben 1618 elementi. Questo dato è la conferma a quanto accenato per la CNN, il dataset originale è fortemente condizionato dalle scelte dell'esperto che ha effettuato la validazione degli esempi.

# References

<a name="cite-schmidhuber201585"/><sup>[^](#ref-1) </sup>Jürgen Schmidhuber. 2015. _Deep learning in neural networks: An overview_. [URL](http://www.sciencedirect.com/science/article/pii/S0893608014002135)

<a name="cite-hinton2006reducing"/><sup>[^](#ref-2) </sup>Hinton, Geoffrey E and Salakhutdinov, Ruslan R. 2006. _Reducing the dimensionality of data with neural networks_.

